Each plant creates one part. For each part, determine a random number of materials it creates the part with. Then, generate a random list of distributors that provide that material.

In [1]:
import arcpy
import pandas as pd
import arcgis
from pprint import pprint
import random

In [2]:
plantFC = r"C:\Users\tim10393\Documents\ArcGIS\Projects\MyProject3\SupplyChain.gdb\ProcessingPlants"
distFC = r"C:\Users\tim10393\Documents\ArcGIS\Projects\MyProject3\SupplyChain.gdb\Distributors"

distSdf = pd.DataFrame.spatial.from_featureclass(distFC)
plantSdf = pd.DataFrame.spatial.from_featureclass(plantFC)

In [3]:
plantFields = ["Name", "Material", "Part", "Distributors"]
def addMatAndDist(inPlantFC, fields):
    matTypes = list(distSdf['Material'].unique())

    uCur = arcpy.da.UpdateCursor(inPlantFC, fields)

    def removeChars(arg1):
        strFromList = str(arg1)
        for char in ['[',']',"'"]:
            strFromList = strFromList.replace(char,"")
        return strFromList

    for row in uCur:
        num = int(random.choice([1,2,3,4])) # get random number
        mats = random.sample(matTypes, num) # get that random number of materials
        part = row[2]
        distList = []
        for mat in mats:
            subDF = distSdf.loc[distSdf['Material']==mat]['Name']
            matDistList = subDF.to_list() # list of distributors providing the materials
            distList.append(random.sample(matDistList, random.randint(1, len(subDF)-1))) # select a random number of distributors from each material used.
        
        flat_list = removeChars(list(set([x for xs in distList for x in xs]))) # turn the list of lists into a list
        
        matsStr = removeChars(list(set(mats)))

        print(flat_list)
        print(matsStr)
            
        print(f"Creating {part} from these {num} materials: {matsStr}. \nMaterials are from these distributors: {flat_list}\n")
        row[1] = matsStr
        row[3] = flat_list
        uCur.updateRow(row)
    #print("Done")
    del uCur
addMatAndDist(plantFC, plantFields)

Distributor 16, Distributor 5, Distributor 19, Distributor 13, Distributor 22, Distributor 17, Distributor 8, Distributor 1
Aluminum, Steel
Creating Part 1 from these 2 materials: Aluminum, Steel. 
Materials are from these distributors: Distributor 16, Distributor 5, Distributor 19, Distributor 13, Distributor 22, Distributor 17, Distributor 8, Distributor 1

Distributor 4, Distributor 11
Plastics
Creating Part 2 from these 1 materials: Plastics. 
Materials are from these distributors: Distributor 4, Distributor 11

Distributor 16, Distributor 11, Distributor 9, Distributor 4, Distributor 13, Distributor 22, Distributor 17, Distributor 14, Distributor 1, Distributor 2
Aluminum, Steel, Plastics
Creating Part 3 from these 3 materials: Aluminum, Steel, Plastics. 
Materials are from these distributors: Distributor 16, Distributor 11, Distributor 9, Distributor 4, Distributor 13, Distributor 22, Distributor 17, Distributor 14, Distributor 1, Distributor 2

Distributor 20, Distributor 21, Di

In [4]:
# use a search cursor to see if each distributor is in the list
def getUniqueDists(inPlantFC, field):
    src = arcpy.da.SearchCursor(inPlantFC, field)
    usedDists = []
    for row in src:
        for d in row[0].split(','):
            usedDists.append(d.strip())
    return(usedDists)

#for d in set(getUniqueDists(plantFC, plantFields)):


In [5]:
addMatAndDist(plantFC, plantFields)

d = set(getUniqueDists(distFC, "Name"))
p = set(getUniqueDists(plantFC, "Distributors"))

test = len(d) != len(p)

while d!= p:
    addMatAndDist(plantFC, plantFields)
    p = set(getUniqueDists(plantFC, "Distributors"))
    print(len(p))
    if d == p:
        print("They match")
        break

Distributor 20, Distributor 3, Distributor 21, Distributor 23, Distributor 19, Distributor 13, Distributor 6, Distributor 10, Distributor 17, Distributor 14, Distributor 7, Distributor 18, Distributor 15
Aluminum, Metals, Steel, Plastics
Creating Part 1 from these 4 materials: Aluminum, Metals, Steel, Plastics. 
Materials are from these distributors: Distributor 20, Distributor 3, Distributor 21, Distributor 23, Distributor 19, Distributor 13, Distributor 6, Distributor 10, Distributor 17, Distributor 14, Distributor 7, Distributor 18, Distributor 15

Distributor 20, Distributor 3, Distributor 11, Distributor 9, Distributor 4, Distributor 2
Aluminum, Metals, Plastics
Creating Part 2 from these 3 materials: Aluminum, Metals, Plastics. 
Materials are from these distributors: Distributor 20, Distributor 3, Distributor 11, Distributor 9, Distributor 4, Distributor 2

Distributor 19, Distributor 7, Distributor 17, Distributor 1
Aluminum, Steel
Creating Part 3 from these 2 materials: Aluminu